##### Investment Strategy

Date: Jan 25, 2022
Authors: af, jk, jv, gl

In [1]:
import pandas as pd
import numpy as np
import time
import statsmodels.api as sm
from tqdm import tqdm
import os
import math

In [2]:
#%%############################################################################
# Step 1: Preparing the CRSP file
###############################################################################
print("Prepare CRSP file")
t = time.time() # record the current time, so we can measure how long the code takes to run

# load data
crsp = pd.read_csv('crsp.csv')

# Have a look at the data
print(crsp.head())
print(crsp.dtypes)


### formatting ###
# make all variable names lowercase
crsp.columns = map(str.lower,crsp.columns)

# You should see that one of the important variables 'RET' (return) is not a number but 'object'.
# It is preferable to have this variable as a number, which Python denotes as float64 (float64 is just a special way of saying that a variable is a number)
# If you are interested search for 'floating point number'on internet. But it is computer-science issue!

# Changes the returns to number format. Non-numeric data will be NAN
crsp['ret'] = pd.to_numeric(crsp['ret'],errors='coerce') 

# Change the dateformat
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y%m%d')

# Create separate 'year' and 'month' variables (we will use them later to merge CRSP with Compustat)
crsp['year'] = crsp['date'].apply(lambda date: date.year)
crsp['month'] = crsp['date'].apply(lambda date: date.month)

# Calculate market cap
crsp['mktcap'] = crsp['shrout'] * crsp['prc'].abs()


### Some basic data cleaning ###
# keep only common shares
crsp = crsp[crsp['shrcd'].isin([10,11])]

# keep only stocks from NYSE, AMEX and NASDAQ
crsp = crsp[crsp['exchcd'].isin([1,2,3])]

# make sure that there are no duplicates
# usually, we would investigate why there are duplicates and then decide which observation we want to keep
#    For here, it is enough to simply drop the duplicates.
crsp = crsp.drop_duplicates(subset=['date','permno'])


print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block


Prepare CRSP file


D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


   PERMNO      date  SHRCD  EXCHCD SICCD TICKER                     COMNAM  \
0   10000  19851231    NaN     NaN   NaN    NaN                        NaN   
1   10000  19860131   10.0     3.0  3990  OMFGA  OPTIMUM MANUFACTURING INC   
2   10000  19860228   10.0     3.0  3990  OMFGA  OPTIMUM MANUFACTURING INC   
3   10000  19860331   10.0     3.0  3990  OMFGA  OPTIMUM MANUFACTURING INC   
4   10000  19860430   10.0     3.0  3990  OMFGA  OPTIMUM MANUFACTURING INC   

      PRC        RET  SHROUT  
0     NaN        NaN     NaN  
1 -4.3750          C  3680.0  
2 -3.2500  -0.257143  3680.0  
3 -4.4375   0.365385  3680.0  
4 -4.0000  -0.098592  3793.0  
PERMNO      int64
date        int64
SHRCD     float64
EXCHCD    float64
SICCD      object
TICKER     object
COMNAM     object
PRC       float64
RET        object
SHROUT    float64
dtype: object
Completed in 16.7s


In [3]:
crsp['industry'] = crsp.siccd.astype(str).str[:2]
crsp1=crsp[crsp['industry']=='20']
crsp2=crsp[crsp['industry']=='36']
crsp3=crsp[crsp['industry']=='45']
crsp4=crsp[crsp['industry']=='37']
crsp5=crsp[crsp['industry']=='60']

In [4]:
#%%############################################################################
# Step 2: Preparing the Compustat (CCM) file
###############################################################################
print("Prepare Compustat file")
t = time.time() # reset our timer

ccm = pd.read_csv('compustat.csv')

# Have a look at the data
print(ccm.head())
print(ccm.dtypes)


### formatting ###
# make all variable names lowercase
ccm.columns = map(str.lower,ccm.columns)

# Change the dateformat 
ccm['datadate'] = pd.to_datetime(ccm['datadate'], format='%Y%m%d')

# Create separate 'year' and 'month' variables
ccm['year'] = ccm['datadate'].apply(lambda x: x.year)
ccm['month'] = ccm['datadate'].apply(lambda x: x.month)
ccm['debt'] = ccm['at']-ccm['ceq']
ccm['debt_to_equity'] = ccm['debt'] / ccm['ceq']
### Some basic data cleaning ###
# make sure that there are no duplicates (same as above)
ccm = ccm.drop_duplicates(subset=['datadate','gvkey'])
ccm = ccm.drop_duplicates(subset=['year','gvkey'])
ccm = ccm.drop_duplicates(subset=['year','lpermno'])

### Calculate the variables we will use for sorting ###
# Create lagged asset variable 
# Note 1) Pandas does not know the panel data structure, so we need to make sure that the previous
#    record belongs to the same gvkey, and that there are no gaps in the data
# Note 2) We can use the backslash "\" do break long lines
ccm = ccm.sort_values(['gvkey','datadate']) # sort data by gvkey and date

ccm['equity_lagged'] = ccm['ceq'].shift(1) # take the previous record
ccm.loc[(ccm['gvkey'].shift(1) != ccm['gvkey']) | \
        (ccm['year'].shift(1) != ccm['year']-1) | \
        (ccm['month'].shift(1) != ccm['month']),'equity_lagged'] = np.NAN # only use the previous record if it 1) belongs to the same gvkey and 2) is one year older

# cash flow over lagged assets
ccm['debt_to_equity'] = ccm['debt'] / ccm['equity_lagged']

# change in total assets
ccm['change_in_equity'] = ccm['ceq'] / ccm['equity_lagged'] - 1

# It is useful to know how many observations are missing
print('Fraction of observations missing:')
print(1 - ccm.count() / len(ccm))


print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block


Prepare Compustat file
   GVKEY  LPERMNO    LINKDT LINKENDDT  datadate   fyear indfmt consol popsrc  \
0   1001    10015  19830920  19860731  19831231  1983.0   INDL      C      D   
1   1001    10015  19830920  19860731  19841231  1984.0   INDL      C      D   
2   1001    10015  19830920  19860731  19851231  1985.0   INDL      C      D   
3   1003    10031  19831207  19890816  19831231  1983.0   INDL      C      D   
4   1003    10031  19831207  19890816  19841231  1984.0   INDL      C      D   

  datafmt    tic                     conm curcd      at     ceq  dt costat  \
0     STD  AMFD.  A & M FOOD SERVICES INC   USD  14.080   7.823 NaN      I   
1     STD  AMFD.  A & M FOOD SERVICES INC   USD  16.267   8.962 NaN      I   
2     STD  AMFD.  A & M FOOD SERVICES INC   USD  39.495  13.014 NaN      I   
3     STD   ANTQ    A.A. IMPORTING CO INC   USD   8.529   6.095 NaN      I   
4     STD   ANTQ    A.A. IMPORTING CO INC   USD   8.241   6.482 NaN      I   

    sic  
0  5812  
1  5812

In [5]:
ccm.head()

,gvkey,lpermno,linkdt,linkenddt,datadate,fyear,indfmt,consol,popsrc,datafmt,...,ceq,dt,costat,sic,year,month,debt,debt_to_equity,equity_lagged,change_in_equity
0,1001,10015,19830920,19860731,1983-12-31,1983.0,INDL,C,D,STD,...,7.823,NaN,I,5812,1983,12,6.257,NaN,NaN,NaN
1,1001,10015,19830920,19860731,1984-12-31,1984.0,INDL,C,D,STD,...,8.962,NaN,I,5812,1984,12,7.305,0.933785,7.823,0.145596
2,1001,10015,19830920,19860731,1985-12-31,1985.0,INDL,C,D,STD,...,13.014,NaN,I,5812,1985,12,26.481,2.954809,8.962,0.452131
3,1003,10031,19831207,19890816,1983-12-31,1983.0,INDL,C,D,STD,...,6.095,NaN,I,5712,1983,12,2.434,NaN,NaN,NaN
4,1003,10031,19831207,19890816,1984-12-31,1984.0,INDL,C,D,STD,...,6.482,NaN,I,5712,1984,12,1.759,0.288597,6.095,0.063495


In [6]:
ccm['industry'] = ccm.sic.astype(str).str[:2]
ccm1=ccm[ccm['industry']=='20']
ccm2=ccm[ccm['industry']=='36']
ccm3=ccm[ccm['industry']=='45']
ccm4=ccm[ccm['industry']=='37']
ccm5=ccm[ccm['industry']=='60']

In [7]:
#%%############################################################################
# Step 3: Sort stocks into portfolios and calculate returns
###############################################################################
print("Create portfolios")
t = time.time() # reset our timer

# loop over all years in the data
# Note: the first loop loops over the years in range(1981,2017).
#    You can wrap any list by the tqdm command to display a progress bar while looping over the list
portfolios = []
portfolios1 = []# create an empty list to collect the portfolio returns
portfolios2 = []
portfolios3 = []
portfolios4 = []
portfolios5 = []

for year in tqdm(range(1982,2021),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(crsp[crsp['year']==year-1]['permno'].unique())
    permno_list1=list(crsp1[crsp1['year']==year-1]['permno'].unique()) 
    permno_list2=list(crsp2[crsp2['year']==year-1]['permno'].unique())
    permno_list3=list(crsp3[crsp3['year']==year-1]['permno'].unique())
    permno_list4=list(crsp4[crsp4['year']==year-1]['permno'].unique())
    permno_list5=list(crsp5[crsp5['year']==year-1]['permno'].unique())
    
    
    # get the sorting variable for these companies at t-1
    sorting_data = ccm.loc[(ccm['year']==(year-1)) & \
                           (ccm['lpermno'].isin(permno_list)), \
                           ['gvkey','lpermno','debt_to_equity']]
    sorting_data1 = ccm1.loc[(ccm1['year']==(year-1)) & \
                           (ccm1['lpermno'].isin(permno_list1)), \
                           ['gvkey','lpermno','debt_to_equity']]
    sorting_data2 = ccm2.loc[(ccm2['year']==(year-1)) & \
                           (ccm2['lpermno'].isin(permno_list2)), \
                           ['gvkey','lpermno','debt_to_equity']]
    sorting_data3 = ccm3.loc[(ccm3['year']==(year-1)) & \
                           (ccm3['lpermno'].isin(permno_list3)), \
                           ['gvkey','lpermno','debt_to_equity']]
    sorting_data4 = ccm4.loc[(ccm4['year']==(year-1)) & \
                           (ccm4['lpermno'].isin(permno_list4)), \
                           ['gvkey','lpermno','debt_to_equity']]
    sorting_data5 = ccm5.loc[(ccm5['year']==(year-1)) & \
                           (ccm5['lpermno'].isin(permno_list5)), \
                           ['gvkey','lpermno','debt_to_equity']]
    
    # sort into 5 baskets by cashflow over assets
    nportfolios = 5 # number of portfolios
    sorting_data['rank'] = pd.qcut(sorting_data['debt_to_equity'],nportfolios, labels=False)
    sorting_data1['rank'] = pd.qcut(sorting_data1['debt_to_equity'],nportfolios, labels=False)
    sorting_data2['rank'] = pd.qcut(sorting_data2['debt_to_equity'],nportfolios, labels=False)
    sorting_data3['rank'] = pd.qcut(sorting_data3['debt_to_equity'],nportfolios, labels=False)
    sorting_data4['rank'] = pd.qcut(sorting_data4['debt_to_equity'],nportfolios, labels=False)
    sorting_data5['rank'] = pd.qcut(sorting_data5['debt_to_equity'],nportfolios, labels=False)
    
    # select the return data with some time lag to make sure that the accounting information is public (data from July at year t to June in year t+1)
    crsp_window = crsp[((crsp['year']==year) & (crsp['month']>=6)) | \
                       ((crsp['year']==year+1) & (crsp['month']<=6))]
    crsp_window1 = crsp1[((crsp1['year']==year) & (crsp1['month']>=6)) | \
                       ((crsp1['year']==year+1) & (crsp1['month']<=6))]
    crsp_window2 = crsp2[((crsp2['year']==year) & (crsp2['month']>=6)) | \
                       ((crsp2['year']==year+1) & (crsp2['month']<=6))]
    crsp_window3 = crsp3[((crsp3['year']==year) & (crsp3['month']>=6)) | \
                       ((crsp3['year']==year+1) & (crsp3['month']<=6))]
    crsp_window4 = crsp4[((crsp4['year']==year) & (crsp4['month']>=6)) | \
                       ((crsp4['year']==year+1) & (crsp4['month']<=6))]
    crsp_window5 = crsp5[((crsp5['year']==year) & (crsp5['month']>=6)) | \
                       ((crsp5['year']==year+1) & (crsp5['month']<=6))]
    
    # create the portfolio returns for the current window and collect them in portfolios_window
    portfolios_window = []
    portfolios_window1 = []
    portfolios_window2 = []
    portfolios_window3 = []
    portfolios_window4 = []
    portfolios_window5 = []
    
    for p in range(nportfolios):
        # get list of permnos that are in this portfolio
        basket = sorting_data.loc[sorting_data['rank'] == p,'lpermno'].tolist()
        basket1 = sorting_data1.loc[sorting_data1['rank'] == p,'lpermno'].tolist()
        basket2 = sorting_data2.loc[sorting_data2['rank'] == p,'lpermno'].tolist()
        basket3 = sorting_data3.loc[sorting_data3['rank'] == p,'lpermno'].tolist()
        basket4 = sorting_data4.loc[sorting_data4['rank'] == p,'lpermno'].tolist()
        basket5 = sorting_data5.loc[sorting_data5['rank'] == p,'lpermno'].tolist()
        
        # get returns of these permnos
        crsp_p_firms = crsp_window[crsp_window['permno'].isin(basket)]
        crsp_p_firms1 = crsp_window1[crsp_window1['permno'].isin(basket1)]
        crsp_p_firms2 = crsp_window2[crsp_window2['permno'].isin(basket2)]
        crsp_p_firms3 = crsp_window3[crsp_window3['permno'].isin(basket3)]
        crsp_p_firms4 = crsp_window4[crsp_window4['permno'].isin(basket4)]
        crsp_p_firms5 = crsp_window5[crsp_window5['permno'].isin(basket5)]
        
        # pivot returns
        returns = crsp_p_firms.pivot(index='date', columns='permno', values='ret')
        returns = returns.iloc[1:,:] # drop the first row
        returns1 = crsp_p_firms1.pivot(index='date', columns='permno', values='ret')
        returns1 = returns1.iloc[1:,:] # drop the first row
        returns2 = crsp_p_firms2.pivot(index='date', columns='permno', values='ret')
        returns2 = returns2.iloc[1:,:] # drop the first row
        returns3 = crsp_p_firms3.pivot(index='date', columns='permno', values='ret')
        returns3 = returns3.iloc[1:,:] # drop the first row
        returns4 = crsp_p_firms4.pivot(index='date', columns='permno', values='ret')
        returns4 = returns4.iloc[1:,:] # drop the first row
        returns5 = crsp_p_firms5.pivot(index='date', columns='permno', values='ret')
        returns5 = returns5.iloc[1:,:] # drop the first row
        
        # create equally weighted portfolio (monthly rebalancing)
        return_port = returns.mean(axis=1)
        return_port.name = str(p)
        return_port1 = returns1.mean(axis=1)
        return_port1.name = str(p)
        return_port2 = returns2.mean(axis=1)
        return_port2.name = str(p)
        return_port3 = returns3.mean(axis=1)
        return_port3.name = str(p)
        return_port4 = returns4.mean(axis=1)
        return_port4.name = str(p)
        return_port5 = returns5.mean(axis=1)
        return_port5.name = str(p)
        
        # collect portfolio returns in dec_port
        portfolios_window += [return_port]
        portfolios_window1 += [return_port1]
        portfolios_window2 += [return_port2]
        portfolios_window3 += [return_port3]
        portfolios_window4 += [return_port4]
        portfolios_window5 += [return_port5]
        
    # merge the portfolios
    portfolios_window = pd.concat(portfolios_window,axis=1)
    portfolios_window1 = pd.concat(portfolios_window1,axis=1)
    portfolios_window2 = pd.concat(portfolios_window2,axis=1)
    portfolios_window3 = pd.concat(portfolios_window3,axis=1)
    portfolios_window4 = pd.concat(portfolios_window4,axis=1)
    portfolios_window5 = pd.concat(portfolios_window5,axis=1)
        
    # collect results in portfolios
    portfolios += [portfolios_window]
    portfolios1 += [portfolios_window1]
    portfolios2 += [portfolios_window2]
    portfolios3 += [portfolios_window3]
    portfolios4 += [portfolios_window4]
    portfolios5 += [portfolios_window5]

# merge the returns from all windows
portfolios = pd.concat(portfolios,axis=0)
portfolios1 = pd.concat(portfolios1,axis=0)
portfolios2 = pd.concat(portfolios2,axis=0)
portfolios3 = pd.concat(portfolios3,axis=0)
portfolios4 = pd.concat(portfolios4,axis=0)
portfolios5 = pd.concat(portfolios5,axis=0)

print('Step 3 completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block



Create portfolios


years: 100%|███████████████████████████████████████████████████████████████████████████| 39/39 [00:05<00:00,  7.68it/s]

Step 3 completed in 5.1s


In [8]:
# Calculate the average return, std, skewness and kurtosis for each portfolio
average_return= portfolios.mean(axis=0)
monthly_std = portfolios.std()
monthly_skewness = portfolios.skew()
monthly_kurtosis = portfolios.kurtosis()
result = pd.concat([monthly_std, monthly_skewness, monthly_kurtosis,average_return], axis =1)
result.rename(columns = {0: 'monthly_std', 1: 'monthly_skewness', 2:'monthly_kurtosis', 3:'average_return'}, inplace = True)
result

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.067114,0.161036,3.026708,0.014987
1,0.062324,-0.166945,2.829178,0.015110
2,0.057956,-0.504816,3.725402,0.013740
3,0.053571,-0.497199,4.377442,0.012337
4,0.052050,-0.785769,4.517753,0.011202


In [9]:
# Calculate the average return, std, skewness and kurtosis for each portfolio
average_return1= portfolios1.mean(axis=0)
monthly_std1 = portfolios1.std()
monthly_skewness1 = portfolios1.skew()
monthly_kurtosis1 = portfolios1.kurtosis()
result1 = pd.concat([monthly_std1, monthly_skewness1, monthly_kurtosis1,average_return1], axis =1)
result1.rename(columns = {0: 'monthly_std', 1: 'monthly_skewness', 2:'monthly_kurtosis', 3:'average_return'}, inplace = True)
result1

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.046502,-0.380565,2.808583,0.010889
1,0.050832,-0.163154,3.143517,0.015302
2,0.047455,-0.088414,1.121194,0.015775
3,0.048203,-0.376439,2.666032,0.013187
4,0.052578,0.355770,3.860348,0.012769


In [10]:
# Calculate the average return, std, skewness and kurtosis for each portfolio
average_return2= portfolios2.mean(axis=0)
monthly_std2 = portfolios2.std()
monthly_skewness2 = portfolios2.skew()
monthly_kurtosis2 = portfolios2.kurtosis()
result2 = pd.concat([monthly_std2, monthly_skewness2, monthly_kurtosis2,average_return2], axis =1)
result2.rename(columns = {0: 'monthly_std', 1: 'monthly_skewness', 2:'monthly_kurtosis', 3:'average_return'}, inplace = True)
result2

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.086868,1.103012,6.771475,0.017008
1,0.091062,1.977756,17.588459,0.017360
2,0.080778,0.197759,2.352563,0.016328
3,0.085431,-0.027707,1.936908,0.017371
4,0.091723,0.661649,4.264857,0.013826


In [11]:
# Calculate the average return, std, skewness and kurtosis for each portfolio
average_return3= portfolios3.mean(axis=0)
monthly_std3 = portfolios3.std()
monthly_skewness3 = portfolios3.skew()
monthly_kurtosis3 = portfolios3.kurtosis()
result3 = pd.concat([monthly_std3, monthly_skewness3, monthly_kurtosis3,average_return3], axis =1)
result3.rename(columns = {0: 'monthly_std', 1: 'monthly_skewness', 2:'monthly_kurtosis', 3:'average_return'}, inplace = True)
result3

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.100247,0.489419,2.726940,0.010608
1,0.090232,0.087696,2.130237,0.014665
2,0.095542,0.236309,1.337333,0.011792
3,0.113391,0.265484,2.927735,0.014113
4,0.134866,0.971603,5.554342,0.011275


In [12]:
average_return4= portfolios4.mean(axis=0)
monthly_std4 = portfolios4.std()
monthly_skewness4 = portfolios4.skew()
monthly_kurtosis4 = portfolios4.kurtosis()
result4 = pd.concat([monthly_std4, monthly_skewness4, monthly_kurtosis4,average_return4], axis =1)
result4.rename(columns = {0: 'monthly_std', 1: 'monthly_skewness', 2:'monthly_kurtosis', 3:'average_return'}, inplace = True)
result4

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.073458,0.310676,2.858991,0.011327
1,0.075162,0.558663,5.432746,0.013571
2,0.077116,0.051735,3.547307,0.015834
3,0.076238,0.395003,5.991264,0.012274
4,0.095899,0.312969,6.190092,0.013001


In [13]:
average_return5= portfolios5.mean(axis=0)
monthly_std5 = portfolios5.std()
monthly_skewness5 = portfolios5.skew()
monthly_kurtosis5 = portfolios5.kurtosis()
result5 = pd.concat([monthly_std5, monthly_skewness5, monthly_kurtosis5,average_return5], axis =1)
result5.rename(columns = {0: 'monthly_std', 1: 'monthly_skewness', 2:'monthly_kurtosis', 3:'average_return'}, inplace = True)
result5

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.050386,0.018060,4.977928,0.012991
1,0.056273,-0.178625,2.643950,0.012666
2,0.054147,-0.525159,2.814656,0.014233
3,0.059352,-0.522155,2.887758,0.012028
4,0.065041,0.191766,4.373113,0.011489


In [14]:
#%%############################################################################
# Step 4: Performance Evaluation
# Step 4a: Merge Portfolio returns with Fama French data
###############################################################################

### load and prepare fama french data ###
# load Fama French monthly factors
ff = pd.read_csv('F-F_Research_Data_Factors.csv',skiprows=[0,1,2])
ff = ff.iloc[:1149]
# rename columns
ff.rename({'Mkt-RF':'ExMkt',
           'Unnamed: 0':'date'},axis=1,inplace=True)
ff=ff.astype('float')
# date variables
ff['year'] = ff['date'] // 100
ff['month'] = ff['date'] % 100
ff.set_index('date',inplace=True)


### formatting ###
# FF data is in percent. Convert to simple returns
ff[['ExMkt', 'SMB', 'HML', 'RF']] /= 100


### merge portfolio returns with Fama French data ###
# date variables
portfolios_ff = portfolios.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff['year'] = portfolios_ff.index.year
portfolios_ff['month'] = portfolios_ff.index.month

portfolios_ff1 = portfolios1.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff1['year'] = portfolios_ff1.index.year
portfolios_ff1['month'] = portfolios_ff1.index.month

portfolios_ff2 = portfolios2.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff2['year'] = portfolios_ff2.index.year
portfolios_ff2['month'] = portfolios_ff2.index.month

portfolios_ff3 = portfolios3.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff3['year'] = portfolios_ff3.index.year
portfolios_ff3['month'] = portfolios_ff3.index.month

portfolios_ff4 = portfolios4.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff4['year'] = portfolios_ff4.index.year
portfolios_ff4['month'] = portfolios_ff4.index.month

portfolios_ff5 = portfolios5.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff5['year'] = portfolios_ff5.index.year
portfolios_ff5['month'] = portfolios_ff5.index.month

# merge
portfolios_ff = pd.merge(portfolios_ff,ff,on=['year','month'])
portfolios_ff1 = pd.merge(portfolios_ff1,ff,on=['year','month'])
portfolios_ff2 = pd.merge(portfolios_ff2,ff,on=['year','month'])
portfolios_ff3 = pd.merge(portfolios_ff3,ff,on=['year','month'])
portfolios_ff4 = pd.merge(portfolios_ff4,ff,on=['year','month'])
portfolios_ff5 = pd.merge(portfolios_ff5,ff,on=['year','month'])


In [15]:
ff.head()

,ExMkt,SMB,HML,RF,year,month
date,,,,,,
192607.0,0.0296,-0.0238,-0.0273,0.0022,1926.0,7.0
192608.0,0.0264,-0.0147,0.0414,0.0025,1926.0,8.0
192609.0,0.0036,-0.0139,0.0012,0.0023,1926.0,9.0
192610.0,-0.0324,-0.0013,0.0065,0.0032,1926.0,10.0
192611.0,0.0253,-0.0016,-0.0038,0.0031,1926.0,11.0


In [16]:
#%%############################################################################
# Step 4b: Regressions
###############################################################################

# show average returns (annualized and in percent)
print("Average returns (annualized percent)\n",((1+portfolios1.mean(axis=0))**12-1)*100)
print("Average returns (annualized percent)\n",((1+portfolios2.mean(axis=0))**12-1)*100)

# Calculate the excess returns
for p in range(nportfolios):
    portfolios_ff['ExRet_'+str(p)] = portfolios_ff[str(p)]-portfolios_ff['RF']
    portfolios_ff1['ExRet_'+str(p)] = portfolios_ff1[str(p)]-portfolios_ff1['RF']
    portfolios_ff2['ExRet_'+str(p)] = portfolios_ff2[str(p)]-portfolios_ff2['RF']
    portfolios_ff3['ExRet_'+str(p)] = portfolios_ff3[str(p)]-portfolios_ff3['RF']
    portfolios_ff4['ExRet_'+str(p)] = portfolios_ff4[str(p)]-portfolios_ff4['RF']
    portfolios_ff5['ExRet_'+str(p)] = portfolios_ff5[str(p)]-portfolios_ff5['RF']


### Market model regressions ###
table_capm = []
table_capm20 = []
table_capm36 = []
table_capm45 = []
table_capm37 = []
table_capm60 = []
for p in range(nportfolios):
    # regress portfolio excess return on market excess return
    results_capm = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff['ExMkt'])).fit()
    results20_capm = sm.OLS(portfolios_ff1['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff1['ExMkt'])).fit()
    results36_capm = sm.OLS(portfolios_ff2['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff2['ExMkt'])).fit()
    results45_capm = sm.OLS(portfolios_ff3['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff3['ExMkt'])).fit()
    results37_capm = sm.OLS(portfolios_ff4['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff4['ExMkt'])).fit()
    results60_capm = sm.OLS(portfolios_ff5['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff5['ExMkt'])).fit()
    
    # collect results
    table_row_capm = pd.DataFrame({'alpha':results_capm.params['const'],
                              'beta_mkt':results_capm.params['ExMkt'],
                              'alpha_t':results_capm.tvalues['const'],
                              'p-value':results_capm.pvalues['const'],
                              'rmse':np.sqrt(results_capm.mse_resid),
                              'R2':results_capm.rsquared},
                             index=[p])
    table_capm += [table_row_capm]
    
    table_row20_capm = pd.DataFrame({'alpha':results20_capm.params['const'],
                              'beta_mkt':results20_capm.params['ExMkt'],
                              'alpha_t':results20_capm.tvalues['const'],
                              'p-value':results20_capm.pvalues['const'],
                              'rmse':np.sqrt(results20_capm.mse_resid),
                              'R2':results20_capm.rsquared},
                             index=[p])
    table_capm20 += [table_row20_capm]
    
    table_row36_capm = pd.DataFrame({'alpha':results36_capm.params['const'],
                              'beta_mkt':results36_capm.params['ExMkt'],
                              'alpha_t':results36_capm.tvalues['const'],
                              'p-value':results36_capm.pvalues['const'],
                              'rmse':np.sqrt(results36_capm.mse_resid),
                              'R2':results36_capm.rsquared},
                             index=[p])
    table_capm36 += [table_row36_capm]
    
    table_row45_capm = pd.DataFrame({'alpha':results45_capm.params['const'],
                              'beta_mkt':results45_capm.params['ExMkt'],
                              'alpha_t':results45_capm.tvalues['const'],
                              'p-value':results45_capm.pvalues['const'],
                              'rmse':np.sqrt(results45_capm.mse_resid),
                              'R2':results45_capm.rsquared},
                             index=[p])
    table_capm45 += [table_row45_capm]
    
    table_row37_capm = pd.DataFrame({'alpha':results37_capm.params['const'],
                              'beta_mkt':results37_capm.params['ExMkt'],
                              'alpha_t':results37_capm.tvalues['const'],
                              'p-value':results37_capm.pvalues['const'],
                              'rmse':np.sqrt(results37_capm.mse_resid),
                              'R2':results37_capm.rsquared},
                             index=[p])
    table_capm37 += [table_row37_capm]
    
    table_row60_capm = pd.DataFrame({'alpha':results60_capm.params['const'],
                              'beta_mkt':results60_capm.params['ExMkt'],
                              'alpha_t':results60_capm.tvalues['const'],
                              'p-value':results60_capm.pvalues['const'],
                              'rmse':np.sqrt(results60_capm.mse_resid),
                              'R2':results60_capm.rsquared},
                             index=[p])
    table_capm60 += [table_row60_capm]

# Combine the results for all portfolios
table_capm = pd.concat(table_capm,axis=0)
table_capm.index.name = 'quintile'
table_capm20 = pd.concat(table_capm20,axis=0)
table_capm20.index.name = 'quintile'
table_capm36 = pd.concat(table_capm36,axis=0)
table_capm36.index.name = 'quintile'
table_capm45 = pd.concat(table_capm45,axis=0)
table_capm45.index.name = 'quintile'
table_capm37 = pd.concat(table_capm37,axis=0)
table_capm37.index.name = 'quintile'
table_capm60 = pd.concat(table_capm60,axis=0)
table_capm60.index.name = 'quintile'

# show results
# print("CAPM-20\n",table_capm20)
# print("CAPM-55\n",table_capm54)

### Three Factor model regressions ###
table_ff = []
table_ff20 = []
table_ff36 = []
table_ff45 = []
table_ff37 = []
table_ff60 = []

for p in range(nportfolios):
    # regress portfolio excess return on market excess return
    results_ff = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff[['ExMkt','SMB','HML']])).fit()
    results20_ff = sm.OLS(portfolios_ff1['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff1[['ExMkt','SMB','HML']])).fit()
    results36_ff = sm.OLS(portfolios_ff2['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff2[['ExMkt','SMB','HML']])).fit()
    results45_ff = sm.OLS(portfolios_ff3['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff3[['ExMkt','SMB','HML']])).fit()
    results37_ff = sm.OLS(portfolios_ff4['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff4[['ExMkt','SMB','HML']])).fit()
    results60_ff = sm.OLS(portfolios_ff5['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff5[['ExMkt','SMB','HML']])).fit()
    
    # collect results
    table_row_ff = pd.DataFrame({'alpha':results_ff.params['const'],
                              'beta_mkt':results_ff.params['ExMkt'],
                              'beta_size':results_ff.params['SMB'],
                              'beta_hml':results_ff.params['HML'],
                              'alpha_t':results_ff.tvalues['const'],
                              'p-value':results_ff.pvalues['const'],
                              'rmse':np.sqrt(results_ff.mse_resid),
                              'R2':results_ff.rsquared},
                             index=[p])
    table_ff += [table_row_ff]
    
    table_row20_ff = pd.DataFrame({'alpha':results20_ff.params['const'],
                              'beta_mkt':results20_ff.params['ExMkt'],
                              'beta_size':results20_ff.params['SMB'],
                              'beta_hml':results20_ff.params['HML'],
                              'alpha_t':results20_ff.tvalues['const'],
                              'p-value':results20_ff.pvalues['const'],
                              'rmse':np.sqrt(results20_ff.mse_resid),
                              'R2':results20_ff.rsquared},
                             index=[p])
    table_ff20 += [table_row20_ff]
    
    table_row36_ff = pd.DataFrame({'alpha':results36_ff.params['const'],
                              'beta_mkt':results36_ff.params['ExMkt'],
                              'beta_size':results36_ff.params['SMB'],
                              'beta_hml':results36_ff.params['HML'],
                              'alpha_t':results36_ff.tvalues['const'],
                              'p-value':results36_ff.pvalues['const'],
                              'rmse':np.sqrt(results36_ff.mse_resid),
                              'R2':results36_ff.rsquared},
                             index=[p])
    table_ff36 += [table_row36_ff]
    
    table_row45_ff = pd.DataFrame({'alpha':results45_ff.params['const'],
                              'beta_mkt':results45_ff.params['ExMkt'],
                              'beta_size':results45_ff.params['SMB'],
                              'beta_hml':results45_ff.params['HML'],
                              'alpha_t':results45_ff.tvalues['const'],
                              'p-value':results45_ff.pvalues['const'],
                              'rmse':np.sqrt(results45_ff.mse_resid),
                              'R2':results45_ff.rsquared},
                             index=[p])
    table_ff45 += [table_row45_ff]
    
    table_row37_ff = pd.DataFrame({'alpha':results37_ff.params['const'],
                              'beta_mkt':results37_ff.params['ExMkt'],
                              'beta_size':results37_ff.params['SMB'],
                              'beta_hml':results37_ff.params['HML'],
                              'alpha_t':results37_ff.tvalues['const'],
                              'p-value':results37_ff.pvalues['const'],
                              'rmse':np.sqrt(results37_ff.mse_resid),
                              'R2':results37_ff.rsquared},
                             index=[p])
    table_ff37 += [table_row37_ff]
    
    table_row60_ff = pd.DataFrame({'alpha':results60_ff.params['const'],
                              'beta_mkt':results60_ff.params['ExMkt'],
                              'beta_size':results60_ff.params['SMB'],
                              'beta_hml':results60_ff.params['HML'],
                              'alpha_t':results60_ff.tvalues['const'],
                              'p-value':results60_ff.pvalues['const'],
                              'rmse':np.sqrt(results60_ff.mse_resid),
                              'R2':results60_ff.rsquared},
                             index=[p])
    table_ff60 += [table_row60_ff]


# Combine the results for all portfolios
table_ff = pd.concat(table_ff,axis=0)
table_ff.index.name = 'quintile'
table_ff20 = pd.concat(table_ff20,axis=0)
table_ff20.index.name = 'quintile'
table_ff36 = pd.concat(table_ff36,axis=0)
table_ff36.index.name = 'quintile'
table_ff45 = pd.concat(table_ff45,axis=0)
table_ff45.index.name = 'quintile'
table_ff37 = pd.concat(table_ff37,axis=0)
table_ff37.index.name = 'quintile'
table_ff60 = pd.concat(table_ff60,axis=0)
table_ff60.index.name = 'quintile'


Average returns (annualized percent)
 0    13.878462
1    19.989813
2    20.661999
3    17.023615
4    16.445737
dtype: float64
Average returns (annualized percent)
 0    22.431042
1    22.940809
2    21.453023
3    22.957216
4    17.912240
dtype: float64


D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except f

In [17]:
portfolios_ff['long-short'] = portfolios_ff[table_capm['alpha'].idxmax().astype(str)]-portfolios_ff[table_capm['alpha'].idxmin().astype(str)]

In [18]:
results_capm_ls = sm.OLS(portfolios_ff['long-short'],
                     sm.add_constant(portfolios_ff['ExMkt'])).fit()
    
table_row_capm_ls = pd.DataFrame({'alpha':results_capm_ls.params['const'],
                              'beta_mkt':results_capm_ls.params['ExMkt'],
                              'alpha_t':results_capm_ls.tvalues['const'],
                              'p-value':results_capm_ls.pvalues['const'],
                              'rmse':np.sqrt(results_capm_ls.mse_resid),
                              'R2':results_capm_ls.rsquared},
                             index=[5])
table_capm_ls = pd.concat([table_capm, table_row_capm_ls],axis=0)


results_ff_ls = sm.OLS(portfolios_ff['long-short'],
                     sm.add_constant(portfolios_ff[['ExMkt','SMB','HML']])).fit()

table_row_ff_ls = pd.DataFrame({'alpha':results_ff_ls.params['const'],
                              'beta_mkt':results_ff_ls.params['ExMkt'],
                              'beta_size':results_ff_ls.params['SMB'],
                              'beta_hml':results_ff_ls.params['HML'],
                              'alpha_t':results_ff_ls.tvalues['const'],
                              'p-value':results_ff_ls.pvalues['const'],
                              'rmse':np.sqrt(results_ff_ls.mse_resid),
                              'R2':results_ff_ls.rsquared},
                             index=[5])
table_ff_ls = pd.concat([table_ff, table_row_ff_ls],axis=0)
table_capm_ls

D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,alpha,beta_mkt,alpha_t,p-value,rmse,R2
0,0.002720,1.166342,1.355652,0.175867,0.042716,0.597879
1,0.002800,1.171695,1.739691,0.082574,0.034264,0.699881
2,0.001780,1.128187,1.303026,0.193209,0.029081,0.750080
3,0.001038,1.045770,0.825757,0.409364,0.026756,0.752874
4,0.000684,0.948499,0.475420,0.634710,0.030615,0.656843
5,0.002116,0.223196,1.477682,0.140168,0.030489,0.096551


In [19]:
table_ff_ls

,alpha,beta_mkt,beta_size,beta_hml,alpha_t,p-value,rmse,R2
0,0.003197,0.990886,1.107416,-0.043262,2.494469,0.012961,0.027192,0.837744
1,0.002746,1.041089,0.961087,0.145424,2.915709,0.003721,0.019983,0.898362
2,0.001214,1.045514,0.786193,0.323048,1.480969,0.139294,0.017397,0.910951
3,0.000265,0.993429,0.636369,0.384583,0.316309,0.751910,0.017799,0.891104
4,-0.000734,0.933921,0.579391,0.629486,-0.760110,0.447575,0.020476,0.847156
5,0.003480,0.107168,0.381697,-0.484062,3.201327,0.001462,0.023062,0.485340


In [20]:
result.loc[5] = [portfolios_ff.std().iloc[-1],portfolios_ff.skew().iloc[-1],portfolios_ff.kurtosis().iloc[-1],portfolios_ff.mean(axis=0).iloc[-1]]
result

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.067114,0.161036,3.026708,0.014987
1,0.062324,-0.166945,2.829178,0.015110
2,0.057956,-0.504816,3.725402,0.013740
3,0.053571,-0.497199,4.377442,0.012337
4,0.052050,-0.785769,4.517753,0.011202
5,0.032043,1.211448,6.686254,0.003907


In [21]:

portfolios_ff1['long-short'] = portfolios_ff1[table_capm20['alpha'].idxmax().astype(str)]-portfolios_ff1[table_capm20['alpha'].idxmin().astype(str)]

In [22]:
results20_capm_ls = sm.OLS(portfolios_ff1['long-short'],
                     sm.add_constant(portfolios_ff1['ExMkt'])).fit()
    
table_row20_capm_ls = pd.DataFrame({'alpha':results20_capm_ls.params['const'],
                              'beta_mkt':results20_capm_ls.params['ExMkt'],
                              'alpha_t':results20_capm_ls.tvalues['const'],
                              'p-value':results20_capm_ls.pvalues['const'],
                              'rmse':np.sqrt(results20_capm_ls.mse_resid),
                              'R2':results20_capm_ls.rsquared},
                             index=[5])
table_capm_ls20 = pd.concat([table_capm20, table_row20_capm_ls],axis=0)


results20_ff_ls = sm.OLS(portfolios_ff1['long-short'],
                     sm.add_constant(portfolios_ff1[['ExMkt','SMB','HML']])).fit()

table_row20_ff_ls = pd.DataFrame({'alpha':results20_ff_ls.params['const'],
                              'beta_mkt':results20_ff_ls.params['ExMkt'],
                              'beta_size':results20_ff_ls.params['SMB'],
                              'beta_hml':results20_ff_ls.params['HML'],
                              'alpha_t':results20_ff_ls.tvalues['const'],
                              'p-value':results20_ff_ls.pvalues['const'],
                              'rmse':np.sqrt(results20_ff_ls.mse_resid),
                              'R2':results20_ff_ls.rsquared},
                             index=[5])
table_ff_ls20 = pd.concat([table_ff20, table_row20_ff_ls],axis=0)
table_capm_ls20

D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,alpha,beta_mkt,alpha_t,p-value,rmse,R2
0,0.003043,0.615430,1.719292,0.086225,0.037681,0.347243
1,0.006450,0.740839,3.554267,0.000418,0.038636,0.423043
2,0.007309,0.692624,4.316996,0.000019,0.036049,0.424017
3,0.004813,0.681199,2.743526,0.006313,0.037349,0.398816
4,0.004648,0.649687,2.251720,0.024805,0.043946,0.303551
5,0.004267,0.077194,2.038992,0.042015,0.044552,0.005952


In [23]:
table_ff_ls20

,alpha,beta_mkt,beta_size,beta_hml,alpha_t,p-value,rmse,R2
0,0.002608,0.591314,0.319478,0.211325,1.523109,0.128413,0.036333,0.395737
1,0.005735,0.725989,0.346429,0.324584,3.325117,0.000954,0.036590,0.484764
2,0.006524,0.714364,0.105656,0.320834,3.970891,0.000083,0.034857,0.463802
3,0.004230,0.693274,0.107852,0.242073,2.444576,0.014873,0.036707,0.421807
4,0.004108,0.637892,0.265817,0.245673,2.028110,0.043120,0.042969,0.337030
5,0.003916,0.123050,-0.213822,0.109509,1.888827,0.059538,0.043982,0.035349


In [24]:
result1.loc[5] = [portfolios_ff1.std().iloc[-1],portfolios_ff1.skew().iloc[-1],portfolios_ff1.kurtosis().iloc[-1],portfolios_ff1.mean(axis=0).iloc[-1]]
result1

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.046502,-0.380565,2.808583,0.010889
1,0.050832,-0.163154,3.143517,0.015302
2,0.047455,-0.088414,1.121194,0.015775
3,0.048203,-0.376439,2.666032,0.013187
4,0.052578,0.355770,3.860348,0.012769
5,0.044637,0.043348,0.732485,0.004886


In [25]:

portfolios_ff2['long-short'] = portfolios_ff2[table_capm36['alpha'].idxmax().astype(str)]-portfolios_ff2[table_capm36['alpha'].idxmin().astype(str)]

In [26]:
results36_capm_ls = sm.OLS(portfolios_ff2['long-short'],
                     sm.add_constant(portfolios_ff2['ExMkt'])).fit()
    
table_row36_capm_ls = pd.DataFrame({'alpha':results36_capm_ls.params['const'],
                              'beta_mkt':results36_capm_ls.params['ExMkt'],
                              'alpha_t':results36_capm_ls.tvalues['const'],
                              'p-value':results36_capm_ls.pvalues['const'],
                              'rmse':np.sqrt(results36_capm_ls.mse_resid),
                              'R2':results36_capm_ls.rsquared},
                             index=[5])
table_capm_ls36 = pd.concat([table_capm36, table_row36_capm_ls],axis=0)


results36_ff_ls = sm.OLS(portfolios_ff2['long-short'],
                     sm.add_constant(portfolios_ff2[['ExMkt','SMB','HML']])).fit()

table_row36_ff_ls = pd.DataFrame({'alpha':results36_ff_ls.params['const'],
                              'beta_mkt':results36_ff_ls.params['ExMkt'],
                              'beta_size':results36_ff_ls.params['SMB'],
                              'beta_hml':results36_ff_ls.params['HML'],
                              'alpha_t':results36_ff_ls.tvalues['const'],
                              'p-value':results36_ff_ls.pvalues['const'],
                              'rmse':np.sqrt(results36_ff_ls.mse_resid),
                              'R2':results36_ff_ls.rsquared},
                             index=[5])
table_ff_ls36 = pd.concat([table_ff36, table_row36_ff_ls],axis=0)
table_capm_ls36

D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,alpha,beta_mkt,alpha_t,p-value,rmse,R2
0,0.003671,1.299637,1.204856,0.228870,0.064876,0.444534
1,0.003276,1.392814,1.042168,0.297874,0.066925,0.463449
2,0.002440,1.368439,0.976194,0.329475,0.053212,0.568760
3,0.002726,1.462759,1.046811,0.295730,0.055442,0.581266
4,-0.001157,1.504768,-0.391899,0.695312,0.062853,0.533378
5,0.004828,-0.205130,1.929079,0.054328,0.053289,0.028702


In [27]:
table_ff_ls36

,alpha,beta_mkt,beta_size,beta_hml,alpha_t,p-value,rmse,R2
0,0.005107,1.050094,1.322064,-0.390118,2.220856,0.026843,0.048781,0.687302
1,0.004085,1.177001,1.288001,-0.149512,1.593517,0.111725,0.054382,0.647242
2,0.003195,1.166458,1.206172,-0.138986,1.753946,0.080100,0.038641,0.773573
3,0.003102,1.283905,1.165859,0.003952,1.510753,0.131532,0.043561,0.742618
4,-0.001110,1.322426,1.301238,0.150352,-0.468141,0.639903,0.050314,0.702271
5,0.006217,-0.272332,0.020826,-0.540470,2.593521,0.009800,0.050854,0.119240


In [28]:
result2.loc[5] = [portfolios_ff2.std().iloc[-1],portfolios_ff2.skew().iloc[-1],portfolios_ff2.kurtosis().iloc[-1],portfolios_ff2.mean(axis=0).iloc[-1]]
result2

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.086868,1.103012,6.771475,0.017008
1,0.091062,1.977756,17.588459,0.017360
2,0.080778,0.197759,2.352563,0.016328
3,0.085431,-0.027707,1.936908,0.017371
4,0.091723,0.661649,4.264857,0.013826
5,0.054013,-0.732426,6.864872,0.003182


In [29]:

portfolios_ff3['long-short'] = portfolios_ff3[table_capm45['alpha'].idxmax().astype(str)]-portfolios_ff3[table_capm45['alpha'].idxmin().astype(str)]

In [30]:
results45_capm_ls = sm.OLS(portfolios_ff3['long-short'],
                     sm.add_constant(portfolios_ff3['ExMkt'])).fit()
    
table_row45_capm_ls = pd.DataFrame({'alpha':results45_capm_ls.params['const'],
                              'beta_mkt':results45_capm_ls.params['ExMkt'],
                              'alpha_t':results45_capm_ls.tvalues['const'],
                              'p-value':results45_capm_ls.pvalues['const'],
                              'rmse':np.sqrt(results45_capm_ls.mse_resid),
                              'R2':results45_capm_ls.rsquared},
                             index=[5])
table_capm_ls45 = pd.concat([table_capm45, table_row45_capm_ls],axis=0)


results45_ff_ls = sm.OLS(portfolios_ff3['long-short'],
                     sm.add_constant(portfolios_ff3[['ExMkt','SMB','HML']])).fit()

table_row45_ff_ls = pd.DataFrame({'alpha':results45_ff_ls.params['const'],
                              'beta_mkt':results45_ff_ls.params['ExMkt'],
                              'beta_size':results45_ff_ls.params['SMB'],
                              'beta_hml':results45_ff_ls.params['HML'],
                              'alpha_t':results45_ff_ls.tvalues['const'],
                              'p-value':results45_ff_ls.pvalues['const'],
                              'rmse':np.sqrt(results45_ff_ls.mse_resid),
                              'R2':results45_ff_ls.rsquared},
                             index=[5])
table_ff_ls45= pd.concat([table_ff45, table_row45_ff_ls],axis=0)
table_capm_ls45

D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,alpha,beta_mkt,alpha_t,p-value,rmse,R2
0,-0.001430,1.137809,-0.351720,0.725207,0.086580,0.256181
1,0.002005,1.215336,0.589590,0.555752,0.072398,0.359783
2,-0.001517,1.296319,-0.423647,0.672019,0.076262,0.365571
3,0.000480,1.336488,0.105807,0.915781,0.096684,0.275918
4,-0.002045,1.297558,-0.356558,0.721584,0.122105,0.183806
5,0.004050,-0.082223,0.677706,0.498294,0.127227,0.000832


In [31]:
table_ff_ls45

,alpha,beta_mkt,beta_size,beta_hml,alpha_t,p-value,rmse,R2
0,-0.003278,1.133394,0.650147,0.807223,-0.846124,0.397919,0.082202,0.332380
1,0.000865,1.201333,0.482325,0.508260,0.261876,0.793533,0.070102,0.402334
2,-0.002902,1.272497,0.635252,0.623980,-0.847596,0.397100,0.072640,0.426868
3,-0.001522,1.344845,0.609364,0.862110,-0.348484,0.727635,0.092629,0.338243
4,-0.004454,1.336613,0.523848,1.010274,-0.798497,0.424990,0.118337,0.236689
5,0.005319,-0.135281,-0.041522,-0.502014,0.891137,0.373318,0.126636,0.014348


In [32]:
result3.loc[5] = [portfolios_ff3.std().iloc[-1],portfolios_ff3.skew().iloc[-1],portfolios_ff3.kurtosis().iloc[-1],portfolios_ff3.mean(axis=0).iloc[-1]]
result3

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.100247,0.489419,2.726940,0.010608
1,0.090232,0.087696,2.130237,0.014665
2,0.095542,0.236309,1.337333,0.011792
3,0.113391,0.265484,2.927735,0.014113
4,0.134866,0.971603,5.554342,0.011275
5,0.127144,-1.175122,8.324810,0.003390


In [33]:
portfolios_ff4['long-short'] = portfolios_ff4[table_capm37['alpha'].idxmax().astype(str)]-portfolios_ff4[table_capm37['alpha'].idxmin().astype(str)]

In [34]:
results37_capm_ls = sm.OLS(portfolios_ff4['long-short'],
                     sm.add_constant(portfolios_ff4['ExMkt'])).fit()
    
table_row37_capm_ls = pd.DataFrame({'alpha':results37_capm_ls.params['const'],
                              'beta_mkt':results37_capm_ls.params['ExMkt'],
                              'alpha_t':results37_capm_ls.tvalues['const'],
                              'p-value':results37_capm_ls.pvalues['const'],
                              'rmse':np.sqrt(results37_capm_ls.mse_resid),
                              'R2':results37_capm_ls.rsquared},
                             index=[5])
table_capm_ls37 = pd.concat([table_capm37, table_row37_capm_ls],axis=0)


results37_ff_ls = sm.OLS(portfolios_ff4['long-short'],
                     sm.add_constant(portfolios_ff4[['ExMkt','SMB','HML']])).fit()

table_row37_ff_ls = pd.DataFrame({'alpha':results37_ff_ls.params['const'],
                              'beta_mkt':results37_ff_ls.params['ExMkt'],
                              'beta_size':results37_ff_ls.params['SMB'],
                              'beta_hml':results37_ff_ls.params['HML'],
                              'alpha_t':results37_ff_ls.tvalues['const'],
                              'p-value':results37_ff_ls.pvalues['const'],
                              'rmse':np.sqrt(results37_ff_ls.mse_resid),
                              'R2':results37_ff_ls.rsquared},
                             index=[5])
table_ff_ls37= pd.concat([table_ff37, table_row37_ff_ls],axis=0)
table_capm_ls37

D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,alpha,beta_mkt,alpha_t,p-value,rmse,R2
0,-0.000445,1.104681,-0.173014,0.862716,0.054742,0.448150
1,0.001329,1.163236,0.517429,0.605103,0.054687,0.474313
2,0.003130,1.220877,1.213652,0.225495,0.054906,0.496476
3,-0.000677,1.251672,-0.276796,0.782059,0.052095,0.535149
4,-0.001990,1.505852,-0.616272,0.538016,0.068751,0.488943
5,0.005120,-0.284974,1.810001,0.070940,0.060224,0.042744


In [35]:
table_ff_ls37

,alpha,beta_mkt,beta_size,beta_hml,alpha_t,p-value,rmse,R2
0,-0.001342,1.037803,0.783066,0.452381,-0.579631,0.562445,0.049115,0.557678
1,0.000309,1.102061,0.782876,0.500296,0.134603,0.892984,0.048772,0.583687
2,0.001411,1.186878,0.820582,0.778772,0.650523,0.515676,0.046010,0.647940
3,-0.001940,1.237698,0.523285,0.561712,-0.857127,0.391817,0.048015,0.606813
4,-0.004469,1.520645,0.722535,1.063473,-1.592657,0.111918,0.059534,0.618431
5,0.005880,-0.333767,0.098047,-0.284701,2.093153,0.036878,0.059598,0.066582


In [36]:
result4.loc[5] = [portfolios_ff4.std().iloc[-1],portfolios_ff4.skew().iloc[-1],portfolios_ff4.kurtosis().iloc[-1],portfolios_ff4.mean(axis=0).iloc[-1]]
result4

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.073458,0.310676,2.858991,0.011327
1,0.075162,0.558663,5.432746,0.013571
2,0.077116,0.051735,3.547307,0.015834
3,0.076238,0.395003,5.991264,0.012274
4,0.095899,0.312969,6.190092,0.013001
5,0.061488,-0.463783,1.544422,0.002833


In [37]:

portfolios_ff5['long-short'] = portfolios_ff5[table_capm60['alpha'].idxmax().astype(str)]-portfolios_ff5[table_capm60['alpha'].idxmin().astype(str)]

In [38]:
results60_capm_ls = sm.OLS(portfolios_ff5['long-short'],
                     sm.add_constant(portfolios_ff5['ExMkt'])).fit()
    
table_row60_capm_ls = pd.DataFrame({'alpha':results60_capm_ls.params['const'],
                              'beta_mkt':results60_capm_ls.params['ExMkt'],
                              'alpha_t':results60_capm_ls.tvalues['const'],
                              'p-value':results60_capm_ls.pvalues['const'],
                              'rmse':np.sqrt(results60_capm_ls.mse_resid),
                              'R2':results60_capm_ls.rsquared},
                             index=[5])
table_capm_ls60 = pd.concat([table_capm60, table_row60_capm_ls],axis=0)


results60_ff_ls = sm.OLS(portfolios_ff5['long-short'],
                     sm.add_constant(portfolios_ff5[['ExMkt','SMB','HML']])).fit()

table_row60_ff_ls = pd.DataFrame({'alpha':results60_ff_ls.params['const'],
                              'beta_mkt':results60_ff_ls.params['ExMkt'],
                              'beta_size':results60_ff_ls.params['SMB'],
                              'beta_hml':results60_ff_ls.params['HML'],
                              'alpha_t':results60_ff_ls.tvalues['const'],
                              'p-value':results60_ff_ls.pvalues['const'],
                              'rmse':np.sqrt(results60_ff_ls.mse_resid),
                              'R2':results60_ff_ls.rsquared},
                             index=[5])
table_ff_ls60= pd.concat([table_ff60, table_row60_ff_ls],axis=0)
table_capm_ls60

D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
D:\anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,alpha,beta_mkt,alpha_t,p-value,rmse,R2
0,0.004662,0.675682,2.456176,0.014406,0.040408,0.357993
1,0.003653,0.760786,1.729769,0.084334,0.044964,0.363425
2,0.005394,0.739259,2.677674,0.007675,0.042886,0.372078
3,0.002243,0.857063,1.047968,0.295197,0.045575,0.413580
4,0.001258,0.912668,0.525814,0.599267,0.050947,0.390237
5,0.004135,-0.173409,2.458324,0.014321,0.035816,0.044661


In [39]:
table_ff_ls60

,alpha,beta_mkt,beta_size,beta_hml,alpha_t,p-value,rmse,R2
0,0.002950,0.685515,0.501514,0.734481,1.918182,0.055701,0.032628,0.583213
1,0.001681,0.785851,0.478620,0.833381,0.971924,0.331595,0.036697,0.577804
2,0.003288,0.776569,0.435024,0.880165,2.083296,0.037771,0.033480,0.618954
3,0.000188,0.902492,0.359296,0.850360,0.105651,0.915905,0.037839,0.597507
4,-0.000694,0.956334,0.337581,0.807263,-0.327552,0.743398,0.044925,0.527902
5,0.003981,-0.179765,0.097443,0.072902,2.362979,0.018540,0.035745,0.052527


In [40]:
result5.loc[5] = [portfolios_ff5.std().iloc[-1],portfolios_ff5.skew().iloc[-1],portfolios_ff5.kurtosis().iloc[-1],portfolios_ff5.mean(axis=0).iloc[-1]]
result5

,monthly_std,monthly_skewness,monthly_kurtosis,average_return
0,0.050386,0.018060,4.977928,0.012991
1,0.056273,-0.178625,2.643950,0.012666
2,0.054147,-0.525159,2.814656,0.014233
3,0.059352,-0.522155,2.887758,0.012028
4,0.065041,0.191766,4.373113,0.011489
5,0.036604,-0.411163,10.029044,0.002744
